In [ ]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
from datetime import datetime as dt
import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
from dash import Dash
from dash import dcc
#import dash_core_components as dcc 
import dash_bootstrap_components as dbc
#from dash import dbc
from dash import html
#import dash_html_components as html
from dash.dependencies import Input, Output, State
from datetime import date

In [ ]:
df=pd.read_csv('fykyas.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df['Date']=df['Date'].astype('datetime64')

In [ ]:
df['Date'].head()

In [ ]:
df.info()

In [ ]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
df.columns

In [ ]:
df1= df[['Date','year','month','Month','Week','day','hour','minute','second','Soaking Zone Temp', 'Heating Zone Temp', 'Furnace pressure', 'Soaking Zone Coal Motor RPM','Oxygen %','Air Motor RPM','Combustion Air temp before Rec','Combustion Air temp After Rec','Combustion Air Pressure Before Rec','Combustion Air Pressure After Rec','Coal Consumption per Unit']]

In [ ]:
df1.head()

In [ ]:
df1.shape

In [ ]:
df1_new = df1.groupby(["Date","Month"])['Coal Consumption per Unit'].sum().reset_index().rename(columns={'sum':'Total coal consumption'})#agg(items_ord = ("qty_ordered", "sum"),sales = ("grand_total", "sum")).reset_index()

In [ ]:
df1_new['Date'].head()

In [ ]:
df1_new

In [ ]:
df11=df1.groupby(["Month","year"])['Coal Consumption per Unit'].sum().reset_index()

In [ ]:
df11.shape

In [ ]:
df11

In [ ]:
#fig=None
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = html.Div([
    dbc.Row([
        dbc.Col(html.H1("Fuel Consumption Dashboard", style={'textAlign':'center'}), width=12)
    ]),
    dbc.Row([
        dbc.Col(html.H3("Daily Fuel Consumption"),  width=8),
        dbc.Col(html.H3("Monthly Fuel Consumption"), width=4)
    ]),
    #html.H3("Fuel Consumption Dashboard", style={'textAlign': 'center'}),
    #html.Label("Frequency: ", style={'fontsize':30, 'textalign':'right'}),
    dbc.Row([
        dbc.Col(dcc.DatePickerRange(id='my-date-picker-range',  # ID to be used for callback
                                    calendar_orientation='horizontal',  # vertical or horizontal
                                    day_size=39,  # size of calendar image. Default is 39
                                    end_date_placeholder_text="Return",  # text that appears when no end date chosen
                                    with_portal=False,  # if True calendar will open in a full screen overlay portal
                                    first_day_of_week=0,  # Display of calendar when open (0 = Sunday)
                                    reopen_calendar_on_clear=True,
                                    is_RTL=False,  # True or False for direction of calendar
                                    clearable=True,  # whether or not the user can clear the dropdown
                                    number_of_months_shown=1,  # number of months shown when calendar is open
                                    min_date_allowed=dt(2021, 1, 1),  # minimum date allowed on the DatePickerRange component
                                    max_date_allowed=dt(2021, 12, 31),  # maximum date allowed on the DatePickerRange component
                                    initial_visible_month=dt(2021, 1, 1),  # the month initially presented when the user opens the calendar
                                    start_date=dt(2021, 1, 1).date(),
                                    end_date=dt(2021, 12, 31).date(),
                                    display_format='MMM Do, YY',  # how selected dates are displayed in the DatePickerRange component.
                                    month_format='MMMM, YYYY',  # how calendar headers are displayed when the calendar is opened.
                                    minimum_nights=1,  # minimum number of days between start and end date
                                    persistence=True, persisted_props=['start_date'],persistence_type='session',  # session, local, or memory. Default is 'local'
                                    updatemode='singledate'), xs=4, sm=4, md=4, lg=2, xl=2),  # singledate or bothdates. Determines when callback is triggered),
                
        dbc.Col(dcc.Graph(id='day-bar', figure={},config={'displayModeBar':False}), xs=8, sm=8, md=8, lg=6, xl=6),
        dbc.Col(dcc.Graph(id='month-bar', figure={},config={'displayModeBar': False}), xs=6, sm=6, md=6, lg=4, xl=4)
           ])#, no_gutters=True)

    
])

In [ ]:

@app.callback(
    [Output('day-bar', 'figure'),Output('month-bar', 'figure')],
    [Input('my-date-picker-range', 'start_date'),
     Input('my-date-picker-range', 'end_date')]
)
#fig=None
def update_output(start_date, end_date): # no of arguments = No of inputs
    #global fig
    #dff=[]
    print(start_date)
    print(end_date)
    dff=df1
    mask=(dff['Date'] >= start_date) & (dff['Date'] <= end_date)
    #print(mask)
    dff_selected=dff.loc[mask]
    #print(dff.head())
    df_date=dff_selected.groupby(["Date","Month"])['Coal Consumption per Unit'].sum().reset_index()
    print(df_date.head())
    df_month=dff_selected.groupby(["Month","year"])['Coal Consumption per Unit'].sum().reset_index()
    print(df_month.head())
    figure = px.bar(df_date, x=df_date['Date'], y=df_date['Coal Consumption per Unit'])
    #dff=dff.groupby(['Month'])['Coal Consumption per Unit'].sum()
    figure_month=px.bar(df_month, x=df_month['Month'], y=df_month['Coal Consumption per Unit'])
    return figure, figure_month
    #elif value_of_frequency=='Month':
     #   dff=df11
        
        #dff=df1[df1.groupby(['Month','Date'])['Coal Consumption per Unit'].sum().reset_index()
        #global fig
      #  figure=px.bar(dff, x=dff['Month'], y=dff['Coal Consumption per Unit'])
       # return figure

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)